| title               | tags      | created | modified |
|---------------------|-----------|---------|----------|
| dev-rag-deepdoc-pdf | [dev,rag] |  Sun Mar 16 03:09:57 CST 2025 | Sun Mar 16 03:09:57 CST 2025  |

## 需求：
> 基于ragflow进行二次开发，对pdf文档添加图片解析

# 图片处理与语义增强

## 1. 图片提取

### v1: 使用 `pdfplumber` 提取图片
- **问题**: 提取的图片色域为 CMYK，保存后不是 RGB 通道。
- **尝试**: 使用 Pillow 的 `convert('RGB')` 转换，但效果无效。
- **可能原因**: 需要加载 ICC 或 ICM 文件进行色域转换。

### v2: 改用 `fitz` 提取图片流
- **流程**:
  1. 使用 `pdfplumber` 提取图片 ID。
  2. 将图片 ID 传递给 `fitz`，提取图片流。
- **优势**: 解决了色域转换问题，提高提取效率。
- **问题**: 部分背景图被切割成多个图片

## 2. 图片语义增强
- **背景**: 将图片 URL 嵌入文本中，常规模型在召回时无法理解图片语义。
- **挑战**: 不使用 VL（视觉-语言）模型时，语义理解受限。
- **解决方案**: 使用 VL 模型进行图片增强。
- **当前选择**: 采用 `qwen-vl-max` 模型，提升图片语义理解能力，经测试，qwen-vl-32b-insruct生成的描述效果更好gi

### 2.1 图片增强嵌入

- **方案**:
  - **a. 图片提取时嵌入**
  - **b. 向量化分块前嵌入**

#### 方案a: 图片提取时嵌入
在图片提取时，可以将当前页的每张图片、当前页当前图片前后文本块、上一页内容、下一页内容一并提供给视觉语言（VL）模型。将这些内容视为三个分块，VL 模型将根据上下文生成描述，从而增强图片的语义理解。
参考链接：[Advanced Chunking for PDF/Word with Embedded Images Using Regular Parsers and GPT-4](https://medium.com/@saptarshi701/advanced-chunking-for-pdf-word-with-embedded-images-using-regular-parsers-and-gpt-4o-7f0d5eb97052)

#### 方案b: 向量化分块前嵌入
在完成分块和分词后，可以将当前分块的图片、当前图片前后文本块、上一个分块和下一个分块的内容一起提供给 VL 模型。操作方式与方案a类似，VL 模型根据上下文生成描述。

#### **问题**
对于当前页或分块中存在多个图片的情况，可能需要将这些图片视为“组图”，并综合考虑多个图片之间的关联，以提升描述的准确性和一致性。


## 3. 噪声图片过滤


## 4. 图文分块
